In [13]:
#!pip install instaloader orjson pandas numpy
import glob
import warnings
import pandas as pd
import numpy as np
import os
import re
warnings.filterwarnings("ignore")
import instaloader

def get_instagram_posts(username, startdate, enddate):
    L = instaloader.Instaloader(download_pictures = True, download_videos = True, download_comments= True, compress_json = False)
    L.login("Yourusername" , "Yourpassword")
    profile = instaloader.Profile.from_username(L.context, username)
    posts = profile.get_posts()
    for post in takewhile(lambda p: p.date > startdate, dropwhile(lambda p : p.date > enddate, posts)):
        #print(post.date)
        L.download_post(post, target = profile.username)
import os
import datetime
# instagram username
username = "kraina_07"
# daterange of scraping
startdate = datetime.datetime(2022, 1, 1)
enddate = datetime.datetime.now()
# get your current working directory
current_wkdir = os.curdir
# Call the function. This will automatically store all the scrape data in a folder in your current working directory
get_instagram_posts(username, startdate, enddate)
       
def parse_instafiles(username, path):
    """ 
    This function loads in all the json files generated by the      instaloader package and parses it into a csv file.
    """
    #print('Entering provided directory...')
    os.chdir(os.path.join(path, username))
    
    columns = ['filename', 'datetime', 'type', 'locations_id', 'locations_name', 'mentions', 'hashtags', 'video_duration']
    
    dataframe = pd.DataFrame(columns=[])
    
    #print('Traversing file tree...')
    
    glob('*UTC.json')
    
    for file in glob('*UTC.json'):
        with open(file, 'r') as filecontent:
            filename = filecontent.name
            #print('Found JSON file: ' + filename + '. Loading...')
            
            try:
                metadata = orjson.loads(filecontent.read())
            
            except IOError as e:
                #print("I/O Error. Couldn't load file. Trying the next one...")
                continue
            else:
                pass
            #print('Collecting relevant metadata...')
            time = datetime.datetime.fromtimestamp(int(metadata['node']['taken_at_timestamp']))
            type_ = metadata['node']['__typename']
            likes = metadata['node']['edge_media_preview_like']['count']     
            comments = metadata['node']['edge_media_to_comment']['count']
            username = metadata['node']['owner']['username']
            followers = metadata['node']['owner']['edge_followed_by']['count']
            try:
                text = metadata['node']['edge_media_to_caption']['edges'][0]['node']['text']
            except:
                text = ""
            try:
                post_id = metadata['node']['id']
            except:
                post_id = ""
            minedata = {'filename': filename, 'time': time, 'text': text,
                    'likes': likes, 'comments' : comments, 'username' : username,  'followers' : followers, 'post_id' : post_id}
            #print('Writing to dataframe...')
            dataframe = dataframe.append(minedata, ignore_index=True)
            #print('Closing file...')
            del metadata
            filecontent.close()
    #print('Storing dataframe to CSV file...')
    #print('Done.')
    dataframe['source'] = 'Instagram'
    return dataframe
import pandas as pd
from glob import glob
import json

import orjson
import time
os.chdir("Absolute address of the directory")
df_instagram = parse_instafiles(username, "")
df_instagram.to_csv("instagramposts.csv")


BadCredentialsException: Login error: Wrong password.

NameError: name 'data' is not defined